<a href="https://colab.research.google.com/github/JAKravitz/JPL_WQ_ML/blob/main/TrainingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JAKravitz/JPL_WQ_ML.git

Cloning into 'JPL_WQ_ML'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [2]:
import sys
sys.path.insert(0,'/content/JPL_WQ_ML')

In [10]:
import pickle
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from MLPregressor import MLPregressor
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/nasa_npp/RT/sensorIDX_rrs.p', 'rb') as fp:
  sensorData = pickle.load(fp)

In [6]:
list(sensorData.keys())

['s3', 's2', 'modis', 'meris', 'l8', 'hico']

In [ ]:
hicoData = sensorData['hico']
list(hicoData.keys())

In [8]:
target = ['chl','PC','Cnap']

batch_info = {
              'epochs':10,
              'batch_size':32,
              'lrate':.0001,
              'split':.1,
              'targets': target,
              'cv':4,
              'meta': None,
              'Xpca': None, 
              }

for key, value in batch_info.items():
    value = None if value == 'None' else value
    batch_info[key] = value

In [9]:
model = MLPregressor(batch_info)
X, y, y0 = model.getXY(hicoData)
results = model.prep_results(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=batch_info['split'])
kfold = KFold(n_splits=batch_info['cv'], shuffle=True)
for train, test in kfold.split(X_train, y_train):
    model.build()
    X_tn, X_tt = X_train.iloc[train,:], X_train.iloc[test,:]
    y_tn, y_tt = y_train.iloc[train,:], y_train.iloc[test,:] 
    history = model.fit(X_tn,y_tn)
    y_ht = model.predict(X_tt)
    results = model.evaluate(y_ht,y_tt,results,'cv') 

print ('\n## FINAL MODEL ##\n')
history2 = model.fit(X_train,y_train)
results['train_loss'].append(history2.history['loss'])
results['val_loss'].append(history2.history['val_loss'])
y_hat = model.predict(X_test)
results = model.evaluate(y_hat,y_test,results,'final') 
results['batch_info'] = batch_info

#%% plot
fig, ax = plt.subplots()
ax.scatter(results['agd']['final']['ytest'], results['agd']['final']['yhat'],s=.1,c='k')

NameError: ignored